In [ ]:
!pip install pretty_midi
import os
import numpy as np
import random
import joblib
import pretty_midi
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout, MultiHeadAttention, Flatten
from tensorflow.keras.models import Model, load_model
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping


from google.colab import drive
# Google Drive'ı bağla
drive.mount('/content/drive')

In [ ]:
# Enable mixed precision
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

# Enable XLA JIT compilation for faster execution
tf.config.optimizer.set_jit(True)

# Enable GPU memory growth (avoid full pre-allocation)
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
# Yollar
MODEL_PATH = '/content/drive/MyDrive/transformer_music_model.keras'
DURATION_SCALER_PATH = '/content/drive/MyDrive/duration_scaler.pkl'
VELOCITY_SCALER_PATH = '/content/drive/MyDrive/velocity_scaler.pkl'
INSTRUMENT_SCALER_PATH = '/content/drive/MyDrive/instrument_scaler.pkl'
INSTRUMENT_ENCODER_PATH = '/content/drive/MyDrive/instrument_encoder.pkl'
MIDI_DIR = '/content/drive/MyDrive/midi_files'

In [ ]:
# 1. Veri hazırlama, model eğitimi ve kaydetme
class MusicModel:

    def __init__(self, seq_len=50, max_instr=4):
        self.seq_len = seq_len
        self.max_instr = max_instr
        self.model = None
        self.duration_scaler = MinMaxScaler()
        self.velocity_scaler = MinMaxScaler()
        self.instrument_scaler = MinMaxScaler()
        self.instrument_encoder = LabelEncoder()

    def extract_notes_by_time(self, midi_path, time_resolution=0.1):
        midi = pretty_midi.PrettyMIDI(midi_path)
        max_time = midi.get_end_time()
        time_steps = np.arange(0, max_time, time_resolution)
        note_groups = []  # ERROR:root:Internal Python error in the inspect module.

    def train_and_save(self, midi_dir=MIDI_DIR):
        files = [os.path.join(midi_dir, f) for f in os.listdir(midi_dir) if f.endswith('.mid')][:600]
        all_groups = []
        for f in files:
            all_groups.extend(self.extract_notes_by_time(f))
        data, instrs = [], []
        for grp in all_groups:
            row = [0] * (self.max_instr * 4)
            for i, n in enumerate(grp[:self.max_instr]):
                p, s, e, v, prog = n
                dur = min(e - s, 2.0)
                row[i*4:(i+1)*4] = [p, dur, v, prog]
                instrs.append(prog)
            data.append(row)
        data = np.array(data)

        # Scaler ve encoder
        self.duration_scaler.fit(data[:, 1].reshape(-1, 1))
        self.velocity_scaler.fit(data[:, 2].reshape(-1, 1))
        self.instrument_scaler.fit(data[:, 3].reshape(-1, 1))
        self.instrument_encoder.fit(instrs)  # ERROR:root:Internal Python error in the inspect module.

        # Normalize
        for i in range(self.max_instr):
            data[:, i*4+1] = self.duration_scaler.transform(data[:, i*4+1].reshape(-1, 1)).flatten()
            data[:, i*4+2] = self.velocity_scaler.transform(data[:, i*4+2].reshape(-1, 1)).flatten()
            data[:, i*4+3] = self.instrument_scaler.transform(data[:, i*4+3].reshape(-1, 1)).flatten()

        # Sequans oluştur
        X, y = [], []
        for idx in range(len(data) - self.seq_len):
            X.append(data[idx:idx + self.seq_len])
            y.append(data[idx + self.seq_len])
        X, y = np.array(X), np.array(y)

        # Modeli kur ve eğit
        self.build_model(X.shape[-1])
        self.model.fit(X, y, epochs=30, batch_size=64, validation_split=0.2)

        # Kaydet
        self.model.save(MODEL_PATH)
        joblib.dump(self.duration_scaler, DURATION_SCALER_PATH)
        joblib.dump(self.velocity_scaler, VELOCITY_SCALER_PATH)
        joblib.dump(self.instrument_scaler, INSTRUMENT_SCALER_PATH)
        joblib.dump(self.instrument_encoder, INSTRUMENT_ENCODER_PATH)
        print('Eğitim tamamlandı ve model kaydedildi.')

    def load(self):
        self.model = load_model(MODEL_PATH)
        self.duration_scaler = joblib.load(DURATION_SCALER_PATH)
        self.velocity_scaler = joblib.load(VELOCITY_SCALER_PATH)
        self.instrument_scaler = joblib.load(INSTRUMENT_SCALER_PATH)
        self.instrument_encoder = joblib.load(INSTRUMENT_ENCODER_PATH)
        print('Model ve scaler/encoder yüklendi.')

In [ ]:
# 2. MIDI -> WAV dönüşümü ve analiz

def midi_to_wav(midi_path, wav_path, sf2_path='/usr/share/sounds/sf2/FluidR3_GM.sf2'):
    temp = '/content/temp.wav'
    subprocess.run(['fluidsynth', '-ni', sf2_path, midi_path, '-F', temp, '-r', '44100'], check=True)
    sr, data = wavfile.read(temp)
    wavfile.write(wav_path, sr, data)
    os.remove(temp)
    print(f'WAV dosyası oluşturuldu: {wav_path}')
    return wav_path

def analyze_midi(midi_path):
    midi = pretty_midi.PrettyMIDI(midi_path)
    notes = [n for inst in midi.instruments for n in inst.notes]
    pitches = [n.pitch for n in notes]
    durations = [n.end - n.start for n in notes]
    info = {
        'Total Notes': len(notes),
        'Pitch Range': (min(pitches), max(pitches)) if pitches else (None, None),
        'Avg Duration': float(np.mean(durations)) if durations else 0.0,
        'Instruments': len(midi.instruments),
        'Total Length': midi.get_end_time()
    }
    for k, v in info.items():
        print(f"{k}: {v}")
    return info


In [ ]:
# 3. Kullanıcı girdisi ve müzik üretimi

def get_user_inputs():
    print("Müzik üretim parametrelerini girin:")
    bpm = int(input('BPM (60-200): ') or 120)
    duration = float(input('Süre (saniye): ') or 30)
    rhythm = input("Ritim ('steady', 'syncopated', 'random'): ") or 'random'

    # Enstrüman listesini göster
    print("Kullanılabilir enstrümanlar ve program numaraları:")
    for i in range(128):
        try:
            print(f"{i}: {pretty_midi.program_to_instrument_name(i)}")
        except:
            continue

    instr = input('Enstrüman program numaraları (örn. 0,24,40): ')
    instruments = [int(i) for i in instr.split(',')]
    return bpm, duration, rhythm, instruments

def generate_music(model_obj, bpm, duration, rhythm, instruments, num_steps=300):
    seed = np.random.rand(model_obj.seq_len, model_obj.max_instr * 4)
    midi = pretty_midi.PrettyMIDI()
    tracks = {p: pretty_midi.Instrument(program=p) for p in instruments}
    t = 0
    note_div = {'steady': 2, 'syncopated': 3}.get(rhythm, random.choice([2, 3, 4]))
    step = 60 / bpm / note_div
    last_pitch = -1

    for _ in range(num_steps):
        pred = model_obj.model.predict(seed[np.newaxis], verbose=0)[0]
        pitch = int(np.clip(pred[0], 20, 127))

        if pitch == last_pitch:
            pitch += random.choice([-1, 1])
        last_pitch = pitch

        dur = max(0.1, model_obj.duration_scaler.inverse_transform([[pred[1]]])[0][0])
        vel = int(np.clip(model_obj.velocity_scaler.inverse_transform([[pred[2]]])[0][0], 20, 127))
        iv = model_obj.instrument_scaler.inverse_transform([[pred[3]]])[0][0]

        pred_index = int(iv * len(model_obj.instrument_encoder.classes_))
        pred_index = max(0, min(pred_index, len(model_obj.instrument_encoder.classes_) - 1))
        instr_prog = model_obj.instrument_encoder.classes_[pred_index]

        if instr_prog not in instruments:
            instr_prog = random.choice(instruments)

        note = pretty_midi.Note(velocity=vel, pitch=pitch, start=t, end=t + dur)
        tracks[instr_prog].notes.append(note)
        t += step
        seed = np.vstack([seed[1:], pred])

        if t >= duration:
            break

    for inst in tracks.values():
        midi.instruments.append(inst)

    out_midi = '/content/generated_music.mid'
    midi.write(out_midi)
    print(f'MIDI kaydedildi: {out_midi}')
    return out_midi

In [ ]:
# Model eğitimi ve kaydetme
mm = MusicModel()
mm.train_and_save()

In [ ]:
# Modeli yükleme
mm = MusicModel()
mm.load()

In [ ]:
# Kullanıcıdan parametre al ve müzik üret
bpm, duration, rhythm, instruments = get_user_inputs()
midi_file = generate_music(mm, bpm, duration, rhythm, instruments)

In [ ]:
# Oluşan .mid dosyasını wava dönüştürme
!apt update
!apt install fluidsynth -y
!pip install -q pretty_midi scipy

import subprocess
from scipy.io import wavfile

midi_to_wav(midi_file, '/content/generated_music.wav')
analyze_midi(midi_file)

In [ ]:
# Çıktıyı analiz etme
# Analyze the generated MIDI file and include BPM value
if os.path.exists(midi_file):
    print(f"Analyzing MIDI file: {midi_file}")
    analyze_midi(midi_file)
else:
    print(f"MIDI file not found at {midi_file}. Please ensure music is generated first.")

In [ ]:
# Çıktıyı analiz etme ve gerçek BPM ölçümü
if os.path.exists(midi_file):
    print(f"Analyzing MIDI file: {midi_file}")

    # MIDI'den tempo tahmini
    midi_obj = pretty_midi.PrettyMIDI(midi_file)
    try:
        est_tempo = midi_obj.estimate_tempo()
        print(f"Estimated MIDI Tempo: {est_tempo:.2f} BPM")
    except Exception as e:
        print(f"Tempo tahmin edilemedi: {e}")

    # Analiz bilgileri
    analyze_midi(midi_file)
else:
    print(f"MIDI file not found at {midi_file}. Please ensure music is generated first.")

In [ ]:
# Modelini yükleme

model_path = '/content/drive/MyDrive/transformer_music_model.keras'

# Check if the model file exists before attempting to load
if os.path.exists(model_path):
    print(f"Loading model from {model_path}...")
    # Assuming your MusicModel class has a load method that handles this
    # If not, you might need to explicitly use tf.keras.models.load_model
    try:
        mm = MusicModel() # Create an instance of your model class
        mm.model = load_model(model_path) # Load the Keras model into the instance
        print("Model loaded successfully.")
    except Exception as e:
        print(f"Error loading model: {e}")
        # Handle the error appropriately, maybe try loading just the keras model
        try:
             model = load_model(model_path)
             print("Loaded Keras model directly.")
             # If you loaded directly, you might need to manually load scalers/encoders
             # depending on how your original code uses them.
        except Exception as e2:
            print(f"Could not load Keras model directly either: {e2}")

else:
    print(f"Model file not found at {model_path}")
    print("Please ensure the model file is in the specified Google Drive path.")
    print("Run the training section first if you haven't already.")
